# Problema do Caixeiro Vigilante
## Baseado em dados reais das ocorrências capturadas no sistema iremos encontrar a cobertura de arestas

### 1 - Importando os dados e tratando

In [1]:
#Lendo os dados das ocorrencias vindas de um json, extraído do banco de dados da aplicação
import json
tipos_ocorrencia = dict()
latlng = []
with open('ocorrencias.json') as json_file:
    data = json.load(json_file)
json_formatted = json.dumps(data,ensure_ascii=False,indent=4)
ocorrencias = json.loads(json_formatted)
#Extraindo os tipos e as quantidades de ocorrências
for ocorrencia in ocorrencias:
    if ocorrencia['tipo_ocorrencia'] in tipos_ocorrencia:
        tipos_ocorrencia[ocorrencia['tipo_ocorrencia']] += 1
    else:
        tipos_ocorrencia[ocorrencia['tipo_ocorrencia']] = 1
print(tipos_ocorrencia)

{'ASSALTO': 14, 'OUTROS': 63, 'AMEACAS': 5, 'CONSUMO_DE_DROGAS': 3, 'VEICULOS_ABERTOS': 53, 'ACIDENTES': 17, 'FURTO_DE_TERCEIROS': 1}


### Pesos dos tipos de ocorrência
Existem 19 tipos de ocorrências no SIGOc, iremos atribuir um peso para cada um desses tipos, baseado na sua gravidade, os pesos serão utilizados para a criação de arestas e definir pesos de vértices clusterizados, os pesos variam de 0 a 10

In [2]:
pesos_tipos_ocorrencias = dict()

for tipo in tipos_ocorrencia:
    peso = 0
    if tipo == 'HOMICIDIO':
        peso = 10
    elif tipo == 'ESTUPROS':
        peso = 10
    elif tipo == 'ASSALTO':
        peso = 9
    elif tipo == 'FURTO_DE_VEICULO':
        peso = 9
    elif tipo == 'FURTO_DE_PATRIMONIO':
        peso = 9
    elif tipo == 'FURTO_DE_TERCEIROS':
        peso = 9
    elif tipo == 'CONDUCAO_A_HOSPITAL':
        peso = 9
    elif tipo == 'DETENCAO_A_SUSPEITOS':
        peso = 8
    elif tipo == 'ARROMBAMENTO_DE_VEICULOS':
        peso = 8
    elif tipo == 'ARROMBAMENTO_DE_TERCEIROS':
        peso = 8
    elif tipo == 'CONSUMO_DE_DROGAS':
        peso = 7
    elif tipo == 'ACIDENTES':
        peso = 7
    elif tipo == 'COLISAO_DE_VEICULOS':
        peso = 7
    elif tipo == 'DANOS_AO_PATRIMONIO':
        peso = 7
    elif tipo == 'AMEACAS':
        peso = 6
    elif tipo == 'OUTROS':
        peso = 5
    elif tipo == 'CONSUMO_DE_DROGAS':
        peso = 5 
    elif tipo == 'VEICULOS_ABERTOS':
        peso = 4
    elif tipo == 'LUZES_E_EQUIPAMENTOS_LIGADOS':
        peso = 3
    elif tipo == 'PREDIOS_SALAS_E_JANELAS_ABERTAS':
        peso = 3
    pesos_tipos_ocorrencias[tipo] = peso
        
        
        
#Pesos dos tipos de ocorrências encontradas nos registros       
print(pesos_tipos_ocorrencias)

{'ASSALTO': 9, 'OUTROS': 5, 'AMEACAS': 6, 'CONSUMO_DE_DROGAS': 7, 'VEICULOS_ABERTOS': 4, 'ACIDENTES': 7, 'FURTO_DE_TERCEIROS': 9}


### Quantidade de ocorrências antes da clusterização

In [3]:
print(len(ocorrencias))

156


### 2 - Entendendo um pouco os dados
Um gráfico de barras gerados com os tipos de ocorrências vindas da nossa base de dados, apenas para mostrar um pouco melhor a quantidade e relevância de cada um dos tipos

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

x=tipos_ocorrencia.keys()
y=tipos_ocorrencia.values()
sns.set_context(rc={"figure.figsize": (10,10)})
nd = np.arange(len(tipos_ocorrencia))
width=10
plt.xticks(nd, x,rotation=90)
plt.xlim(-0.5,len(tipos_ocorrencia))
fig = plt.bar(nd, y, color=sns.color_palette("Blues",len(tipos_ocorrencia)))

plt.show()

<Figure size 640x480 with 1 Axes>

### 3 - Visualizando os dados
#### Mapa com todas as ocorrências vindas do banco de dados

In [5]:
#Utilizando a biblioteca folium para o mapa
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=[-5.836905, -35.202219], zoom_start=15)
for ocorrencia in ocorrencias:
    folium.Marker(
        location=[ocorrencia['latitude'], ocorrencia['longitude']],
        popup=ocorrencia['tipo_ocorrencia'],
    ).add_to(m)

m

### Algoritmo de clusterização

In [6]:
import geopy.distance
    
#Roda o algoritmo de clusterização até que nao haja mais vertices no raio de distância
ocorrencias_cluster = ocorrencias
for i in ocorrencias_cluster:
    i['peso'] = 0
    if i['tipo_ocorrencia'] in pesos_tipos_ocorrencias:
        i['peso'] = pesos_tipos_ocorrencias[i['tipo_ocorrencia']]
        
exist_vertice_in_radius = True
while exist_vertice_in_radius:
    exist_vertice_in_radius = False
    for i in ocorrencias_cluster:
        for j in ocorrencias_cluster:
            if i != j:
                if(geopy.distance.geodesic((i['latitude'],i['longitude']), (j['latitude'],j['longitude'])).meters <= 50.0):
                    i['peso'] = i['peso'] + j['peso']
                    ocorrencias_cluster.remove(j)
                    exist_vertice_in_radius = True



### Mapa com as ocorrências depois da clusterização
Com a clusterização, vértices num raio de 50 metros viram apenas um, e esse novo vértice tem um peso, que é soma de todos os pesos dos vértices que se uniram a ele

In [7]:
m2 = folium.Map(location=[-5.836905, -35.202219], zoom_start=15)
for ocorrencia in ocorrencias_cluster:
    folium.Marker(
        location=[ocorrencia['latitude'], ocorrencia['longitude']],
        popup='Peso do vértice: ' + str(ocorrencia['peso']),
    ).add_to(m2)

m2

#### Quantidade de ocorrências depois da clusterização

In [8]:
print(len(ocorrencias_cluster))

68


#### Calcula a distância em metros de dois pontos no mapa

In [9]:
def calc_peso_aresta(i,j):
    return geopy.distance.geodesic((i['latitude'],i['longitude']), (j['latitude'],j['longitude'])).meters

#### Inicializando a matriz de adjacência

In [10]:
matrix = []
for x in range(0,len(ocorrencias_cluster)):
    matrix.append([])
    for y in range(0,len(ocorrencias_cluster)):
        matrix[x].append(0)

#### Calculo da matriz de adjacência

In [11]:
x = 0
for i in ocorrencias_cluster:
    peso_aresta = 0
    y = 0
    for j in ocorrencias_cluster:
        if i != j:
            if geopy.distance.geodesic((i['latitude'],i['longitude']), (j['latitude'],j['longitude'])).meters < 250.0:
                novo_peso = calc_peso_aresta(i,j)
                matrix[x][y] = int(novo_peso)
        y = y+1
    x = x+1



#### Matriz de adjacência calculada

In [12]:
print(matrix)

[[0, 92, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 92, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 56, 0, 0, 0], [92, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 53, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 130, 0, 0, 0, 0, 0, 0, 0, 227, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 127, 0, 0, 0], [0, 0, 0, 0, 0, 225, 232, 0, 0, 233, 0, 0, 0, 188, 0, 0, 0, 246, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 148, 0, 0, 189, 163, 0, 0, 201, 0, 0, 0, 0, 211, 0, 0, 117, 0, 105, 0, 0, 0, 0, 0, 0, 193, 166, 0, 0, 0, 0, 0, 237, 69, 153, 0, 0, 0, 237, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 236, 0, 0, 0, 0, 0, 0, 0, 125, 0, 0, 0, 0, 0, 0, 217, 0, 0, 0, 0, 0, 0, 0, 229, 0, 0, 0, 0, 0, 0, 0, 0, 0, 147, 0, 0, 0, 0, 0, 0, 89, 0, 0, 0, 0, 237, 0, 0, 0, 60, 0, 0, 0, 0, 0, 0, 178, 127, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 135, 238, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 248, 0, 0, 166, 0, 0, 

### Gerando o mapa e as arestas a partir da matriz de adjacência

In [13]:
import folium
import numpy as np
import pandas as pd
from collections import namedtuple
some_map = folium.Map(location=[-5.836905, -35.202219], zoom_start=16)
for i in range(0,len(ocorrencias_cluster)):
    p1 = [ocorrencias_cluster[i]['latitude'],ocorrencias_cluster[i]['longitude']]    
    folium.Marker(location=p1).add_to(some_map)
    
    for j in range(0,len(ocorrencias_cluster)):
        if i != j and matrix[i][j] != 0:
            p2 = [ocorrencias_cluster[j]['latitude'],ocorrencias_cluster[j]['longitude']]
            folium.PolyLine(locations=[p1, p2], color='blue').add_to(some_map)
                    
some_map

### Posicionando os vigilantes
Criando um vigilante para cada aresta, supondo que tem-se vigilantes ilimitados

In [14]:
some_map2 = folium.Map(location=[-5.836905, -35.202219], zoom_start=16)
for i in range(0,len(ocorrencias_cluster)):
    p1 = [ocorrencias_cluster[i]['latitude'],ocorrencias_cluster[i]['longitude']]    
    folium.Marker(location=p1).add_to(some_map2)
    
    for j in range(0,len(ocorrencias_cluster)):
        if i != j and matrix[i][j] != 0:
            p2 = [ocorrencias_cluster[j]['latitude'],ocorrencias_cluster[j]['longitude']]
            folium.PolyLine(locations=[p1, p2], color='blue').add_to(some_map2)
            vigilante = [(ocorrencias_cluster[i]['latitude']+ocorrencias_cluster[j]['latitude'])/2,(ocorrencias_cluster[i]['longitude']+ocorrencias_cluster[j]['longitude'])/2]
#             if 
            folium.Marker(location=vigilante,icon=folium.Icon(color='red',icon='info-sign')).add_to(some_map2)      
some_map2

#### Diminuindo a quantidade de vigilantes até a desejada

In [70]:
# print(ocorrencias_cluster)
vigilantes_peso = []
vigilantes_antigos = []
qtd_vigilantes = 50
qtd_vigilantes_armados = 40
qtd_vigilantes_armados_motorizados = 5
qtd_vigilantes_desarmados = 5

some_map3 = folium.Map(location=[-5.836905, -35.202219], zoom_start=16)
for i in range(0,len(ocorrencias_cluster)):
    p1 = [ocorrencias_cluster[i]['latitude'],ocorrencias_cluster[i]['longitude']]    
    folium.Marker(location=p1).add_to(some_map3)
    
    for j in range(0,len(ocorrencias_cluster)):
        if i != j and matrix[i][j] != 0:
            p2 = [ocorrencias_cluster[j]['latitude'],ocorrencias_cluster[j]['longitude']]
            folium.PolyLine(locations=[p1, p2], color='blue').add_to(some_map3)
            vigilante_postion = [(ocorrencias_cluster[i]['latitude']+ocorrencias_cluster[j]['latitude'])/2,(ocorrencias_cluster[i]['longitude']+ocorrencias_cluster[j]['longitude'])/2]
            if ocorrencias_cluster[i]['peso'] >= ocorrencias_cluster[j]['peso']:
#                 GERANDO VIGILANTES DE ACORDO COM TIPOS
                vigilante = {}
                if len(vigilantes_peso) <= qtd_vigilantes:
                    if len(vigilantes_peso) <= qtd_vigilantes_armados:
                        vigilante['position'] = vigilante_postion
                        vigilante['tipo'] = 'ARMADO'
                        vigilante['peso'] = 9
                        vigilantes_peso.append(vigilante)
                    elif len(vigilantes_peso) >= qtd_vigilantes_armados_motorizados:
                        vigilante['position'] = vigilante_postion
                        vigilante['tipo'] = 'ARMADO_MOTORIZADO'
                        vigilante['peso'] = 10
                        vigilantes_peso.append(vigilante)
                    elif len(vigilantes_peso) >= qtd_vigilantes_desarmados:
                        vigilante['position'] = vigilante_postion
                        vigilante['tipo'] = 'DESARMADO'
                        vigilante['peso'] = 8
                        vigilantes_peso.append(vigilante)
                folium.Marker(location=vigilante_postion,icon=folium.Icon(color='red',icon='info-sign')).add_to(some_map3)
                break

some_map3
print(vigilantes_peso,len(vigilantes_peso),len(ocorrencias_cluster))

[{'position': [-5.8322004, -35.205568833333345], 'tipo': 'ARMADO', 'peso': 9}, {'position': [-5.832470338508515, -35.205046343105295], 'tipo': 'ARMADO', 'peso': 9}, {'position': [-5.840140003387734, -35.1972243321686], 'tipo': 'ARMADO', 'peso': 9}, {'position': [-5.836253516894405, -35.200207447389204], 'tipo': 'ARMADO', 'peso': 9}, {'position': [-5.83660950201308, -35.202033322923], 'tipo': 'ARMADO', 'peso': 9}, {'position': [-5.840140003387734, -35.1972243321686], 'tipo': 'ARMADO', 'peso': 9}, {'position': [-5.83927917656866, -35.19767561776405], 'tipo': 'ARMADO', 'peso': 9}, {'position': [-5.842711771068805, -35.19959717988965], 'tipo': 'ARMADO', 'peso': 9}, {'position': [-5.84163111772131, -35.200592279434204], 'tipo': 'ARMADO', 'peso': 9}, {'position': [-5.83975113391966, -35.19921362400055], 'tipo': 'ARMADO', 'peso': 9}, {'position': [-5.83825786833021, -35.2019484044431], 'tipo': 'ARMADO', 'peso': 9}, {'position': [-5.838718235346415, -35.201840522923], 'tipo': 'ARMADO', 'peso':

#### Vigilantes limitados a 50 baseado no peso, e depois clusterizando basendo num raio de distancia

In [74]:
some_map4 = folium.Map(location=[-5.836905, -35.202219], zoom_start=16)
teste = []
for i in vigilantes_peso:
    teste.append(i)
    
exist_vertice_in_radius2 = True
while exist_vertice_in_radius2:
    exist_vertice_in_radius2 = False
    for i in teste:
        p1 = i['position']
        folium.Marker(location=p1,icon=folium.Icon(color='red',icon='info-sign')).add_to(some_map4)
        for j in teste:
            if i != j:
                if(geopy.distance.geodesic((i['position']), (j['position'])).meters <= 100.0):
                    teste.remove(j)
                    exist_vertice_in_radius2 = True

some_map4


In [75]:
print(len(vigilantes_peso),len(vigilantes_antigos),len(teste))

51 0 29
